# Train and Apply Models

In [1]:
from ML.model_training import (
    train_lstm,
)
from ML.labels import build_video_rating_tables
from ML.splits import single_user_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import math, re, itertools
from ML import utils
import sys
from IPython.display import clear_output
from scipy.stats import pearsonr

Generate all subsets of columns for parameters.

In [11]:
X_train, X_test, arousal_train, arousal_test = single_user_split(
    target="arousal", selected_user=20, k_holdouts=1
)
features = utils.filter_features(X_train.columns, remove_bands=["gamma", "delta"])
arousal_train = pd.Series(
    np.where(arousal_train > 3.8, "high", "low"),
    index=arousal_train.index,
    dtype="string",
)
arousal_test = pd.Series(
    np.where(arousal_test > 3.8, "high", "low"),
    index=arousal_test.index,
    dtype="string",
)


def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)


X_train, arousal_train = balance(X_train, arousal_train, seed=5)
X_test, arousal_test = balance(X_test, arousal_test, seed=5)

print("arousal_train counts:\n", arousal_train.value_counts(dropna=False))
print("arousal_test counts:\n", arousal_test.value_counts(dropna=False))

20 [0]
arousal_train counts:
 low     475
high    475
Name: count, dtype: Int64
arousal_test counts:
 low    98
Name: count, dtype: Int64


## LSTM LOO

In [ ]:
best_model = None
best_acc = 0
best_keep = None

best_lr = 0
best_f1 = 0
bar_len = 30

status = f"Best: index= size= | " f"acc= | f1= | prec= | rec="

results = []
X_train_sub = X_train.loc[:, features]
X_test_sub = X_test.loc[:, features]

n_low = (arousal_train == "low").sum()
n_high = (arousal_train == "high").sum()
for lr in [0.0001]:
    for e in [10]:
        for u in [256]:
            for b_s in [128]:
                lstm, X_test_eval, y_test_eval = train_lstm(
                    X_train_sub,
                    X_test_sub,
                    arousal_train,
                    arousal_test,
                    lr=lr,
                    epochs=e,
                    units=u,
                    batch_size=b_s,
                    bidirectional=False,
                )
                y_prob = lstm.predict(X_test_eval).ravel()
                arousal_pred = (y_prob >= 0.5).astype(int)

                acc = accuracy_score(y_test_eval, arousal_pred)
                f1 = f1_score(y_test_eval, arousal_pred, average="weighted")
                prec = precision_score(y_test_eval, arousal_pred, average="weighted")
                rec = recall_score(y_test_eval, arousal_pred, average="weighted")

                if acc > best_acc:
                    best_acc = acc
                    best_model = lstm
                    best_lr = lr
                    best_e = e
                    best_u = u
                    best_b_s = b_s
                    best_f1 = f1
                    best_arousal_pred = arousal_pred
                    status = (
                        f"Best: "
                        f"acc={acc:.6f} | f1={f1:.6f} | prec={prec:.6f} | rec={rec:.6f} | lr={best_lr} | epochs={best_e} | units={best_u} | batch_size={best_b_s}"
                    )
                    print(status)

print("\nConfusion Matrix (pooled):")
print(confusion_matrix(y_test_eval, best_arousal_pred))

print("\nClassification Report (pooled):")
print(classification_report(y_test_eval, best_arousal_pred, zero_division=0))

print(status)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Best: acc=0.839161 | f1=0.835262 | prec=0.874629 | rec=0.839161 | lr=0.0001 | epochs=10 | units=256 | batch_size=128

Confusion Matrix (pooled):
[[ 98  45]
 [  1 142]]

Classification Report (pooled):
              precision    recall  f1-score   support

         0.0       0.99      0.69      0.81       143
         1.0       0.76      0.99      0.86       143

    accuracy                           0.84       286
   macro avg       0.87      0.84      0.84       286
weighted avg       0.87      0.84      0.84       286

Best: acc=0.839161 | f1=0.835262 | prec=0.874629 | rec=0.839161 | lr=0.0001 | epochs=10 | units=256 | batch_size=128


## Choose participants for training

In [ ]:
# define the subjects you want to use as "folds"
subjects = [1, 3, 15, 19, 20]
videos = [2, 7, 10, 15]


# hyper params
lr_grid = [0.0001]
epochs_grid = [10]
units_grid = [64, 128, 256]
batch_size_grid = [64, 128, 256]

best_lr = None
best_e = None
best_u = None
best_b_s = None
best_cv_acc = -np.inf

for lr in lr_grid:
    for e in epochs_grid:
        for u in units_grid:
            for b_s in batch_size_grid:

                acc_list = []

                for i in subjects:
                    for j in videos:
                        while True:
                            X_train, X_test, arousal_train, arousal_test = single_user_split(
                                target="arousal",
                                k_holdouts=1,
                                selected_user=i,
                                holdout_videos=[j]
                            )
                            features = utils.filter_features(X_train.columns, remove_bands=["gamma", "delta"])
                            X_train = X_train.loc[:, features]
                            X_test = X_test.loc[:, features]
                            arousal_train = pd.Series(
                                np.where(arousal_train > 3.8, "high", "low"),
                                index=arousal_train.index,
                                dtype="string",
                            )
                            arousal_test = pd.Series(
                                np.where(arousal_test > 3.8, "high", "low"),
                                index=arousal_test.index,
                                dtype="string",
                            )

                            # if you want to enforce balance conditions, re-enable these:
                            # c = arousal_test.value_counts()
                            # if c.get("high", 0) == 0 or c.get("low", 0) == 0:
                            #     continue
                            # if c.get("low", 0) * 5 >= arousal_train.value_counts().get("low", 0):
                            #     continue

                            # if you want balancing:
                            # X_train, arousal_train = balance(X_train, arousal_train)
                            # X_test, arousal_test = balance(X_test, arousal_test)

                            break

                        lstm, X_test_eval, y_test_eval = train_lstm(
                            X_train,
                            X_test,
                            arousal_train,
                            arousal_test,
                            lr=lr,
                            epochs=e,
                            units=u,
                            batch_size=b_s,
                            bidirectional=False,
                        )

                        y_prob = lstm.predict(X_test_eval).ravel()
                        arousal_pred = (y_prob >= 0.5).astype(int)

                        acc = accuracy_score(y_test_eval, arousal_pred)
                        print(acc)
                        acc_list.append(float(acc))

                mean_acc = float(np.mean(acc_list)) if acc_list else float("nan")
                print(
                    f"lr={lr}, epochs={e}, units={u}, batch_size={b_s} "
                    f"-> mean CV acc={mean_acc:.4f}"
                )

                if mean_acc > best_cv_acc:
                    best_cv_acc = mean_acc
                    best_lr = lr
                    best_e = e
                    best_u = u
                    best_b_s = b_s

print("\nBest hyperparameters (cross-subject CV):")
print(f"  lr={best_lr}")
print(f"  epochs={best_e}")
print(f"  units={best_u}")
print(f"  batch_size={best_b_s}")
print(f"  mean CV accuracy={best_cv_acc:.4f}")


1 [2]
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.9942196531791907
1 [7]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.9744897959183674
1 [10]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
0.0


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.